In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
device = (
    torch.device("mps") if torch.backends.mps.is_available() else
    torch.device("cuda") if torch.cuda.is_available() else
    torch.device("cpu")
)
print(f"Using device: {device}")

In [ ]:
from dataset import MorphII_Dataset

prepipeline = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_dataset = MorphII_Dataset(csv_file="Dataset/Index/validation.csv", transform=prepipeline)
test_dataset = MorphII_Dataset(csv_file="Dataset/Index/test.csv", transform=prepipeline)

BATCH_SIZE = 128
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
# Encoder: maps image and condition -> latent mean and logvar.
class Encoder(nn.Module):
    def __init__(self, latent_dim=100, condition_dim=1):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            # 128×128 -> 64×64
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.1),
            # 64×64 -> 32×32
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            # 32×32 -> 16×16
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            # 16×16 -> 8×8
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1)
        )
        # 128 x 8 x 8 = 8192 features
        self.fc_mu = nn.Linear(8192 + condition_dim, latent_dim)
        self.fc_logvar = nn.Linear(8192 + condition_dim, latent_dim)

    def forward(self, x, condition):
        batch_size = x.size(0)
        x = self.conv(x)  # shape: (B, 128, 8, 8)
        x = x.view(batch_size, -1)  # flatten to (B, 8192)
        x = torch.cat([x, condition], dim=1)  # shape: (B, 8192+condition_dim)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std

# Decoder: decodes latent vector and condition back to a 128×128 image.
class Decoder(nn.Module):
    def __init__(self, latent_dim=100, condition_dim=1):
        super(Decoder, self).__init__()
        # Map latent vector and condition to 8192 features.
        self.fc = nn.Linear(latent_dim + condition_dim, 8192)
        self.deconv = nn.Sequential(
            # Reshape (B, 128, 8, 8) -> upsample to 16×16
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2,
                               padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            # 16×16 -> 32×32
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2,
                               padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            # 32×32 -> 64×64
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2,
                               padding=1, output_padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.1),
            # 64×64 -> 128×128
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2,
                               padding=1, output_padding=1),
            nn.Tanh()  # Output in [-1, 1]
        )

    def forward(self, z, condition):
        x = torch.cat([z, condition], dim=1)  # shape: (B, latent_dim+condition_dim)
        x = self.fc(x)                        # (B, 8192)
        x = x.view(-1, 128, 8, 8)              # reshape to (B, 128, 8, 8)
        x = self.deconv(x)                    # output: (B, 3, 128, 128)
        return x

class ConditionalVAE(nn.Module):
    def __init__(self, latent_dim=100, condition_dim=1):
        super(ConditionalVAE, self).__init__()
        self.encoder = Encoder(latent_dim, condition_dim)
        self.decoder = Decoder(latent_dim, condition_dim)

    def forward(self, x, condition):
        mu, logvar = self.encoder(x, condition)
        z = reparameterize(mu, logvar)
        recon_x = self.decoder(z, condition)
        return recon_x, mu, logvar

latent_dim = 256
condition_dim = 2
model = ConditionalVAE(latent_dim, condition_dim).to(device)
print(model)

In [ ]:
try:
    model = model.to(memory_format=torch.channels_last)
except Exception as e:
    print("Channels last format not supported:", e)

In [ ]:
def load_checkpoint(model, checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint)
    model.eval()
    print(f"Loaded checkpoint from {checkpoint_path}")

checkpoint_path = "checkpoints/checkpoint_epoch_43.pth"
load_checkpoint(model, checkpoint_path, device)

In [ ]:
model.eval()
with torch.no_grad():
    for i in range(6):
        if i<3:
            img, cond = val_dataset[i]
        else:
            img, cond = test_dataset[i]

        img = img.unsqueeze(0).to(device)
        cond = cond.unsqueeze(0).to(device)
        recon, _, _ = model(img, cond)

        orig_np = (img.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
        recon_np = (recon.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)

        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1)
        plt.imshow(orig_np)
        plt.title("Original")
        plt.axis("off")
        plt.subplot(1, 2, 2)
        plt.imshow(recon_np)
        plt.title("Reconstructed")
        plt.axis("off")
        plt.show()

In [ ]:
def generate_age_variation(model, image, cond, age_values):
    """
    Given an image and its condition, encode it and then decode it
    with varying age conditions. The condition now is assumed to contain
    [normalized_age, gender] and only the age is varied.

    Args:
        model: Trained ConditionalVAE.
        image: A single image tensor (C x H x W).
        cond: Its corresponding condition tensor (age, gender), shape [2].
        age_values: Iterable of new normalized age values.

    Returns:
        List of generated images (tensors).
    """
    model.eval()
    outputs = []
    with torch.no_grad():
        # Prepare the image and condition with batch dimension.
        image = image.unsqueeze(0).to(device)
        cond = cond.unsqueeze(0).to(device)  # shape: (1, 2)

        mu, logvar = model.encoder(image, cond)
        z = reparameterize(mu, logvar)

        # Extract the original gender from the condition.
        orig_gender = cond[0, 1].item()
        for age in age_values:
            # Create a new condition vector with the new age and the same gender.
            new_cond = torch.tensor([[age, orig_gender]], dtype=torch.float32, device=device)
            out = model.decoder(z, new_cond)
            outputs.append(out)
    return outputs

def generate_gender_variation(model, image, cond):
    """
    Given an image and its condition, encode it and then decode it
    with varying
    """
    model.eval()
    outputs = []

    with torch.no_grad():
        # Prepare the image and condition with batch dimension.
        image = image.unsqueeze(0).to(device)
        cond = cond.unsqueeze(0).to(device)

        mu, logvar = model.encoder(image, cond)
        z = reparameterize(mu, logvar)

        # Extract the original age from the condition.
        orig_age = cond[0, 0].item()
        for gender in [0, 1]:
            # Create a new condition vector with the new
            new_cond = torch.tensor([[orig_age, gender]], dtype=torch.float32, device=device)
            out = model.decoder(z, new_cond)
            outputs.append(out)

    return outputs

In [ ]:
sample_img, sample_cond = test_dataset[0]
age_range = np.linspace(0.0, 1.0, 10)
generated_age_images = generate_age_variation(model, sample_img, sample_cond, age_range)

plt.figure(figsize=(15, 3))
for i, gen in enumerate(generated_age_images):
    gen_np = (gen.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
    plt.subplot(1, len(generated_age_images), i+1)
    plt.imshow(gen_np)
    plt.title(f"Age: {age_range[i]:.2f}")
    plt.axis("off")
plt.show()

In [ ]:
sample_img, sample_cond = test_dataset[0]
generated_gender_images = generate_gender_variation(model, sample_img, sample_cond)

plt.figure(figsize=(15, 3))
for i, gen in enumerate(generated_gender_images):
    gen_np = (gen.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
    plt.subplot(1, len(generated_gender_images), i+1)
    plt.imshow(gen_np)
    plt.title(f"Gender: {[0, 1][i]:.2f}")
    plt.axis("off")
plt.show()